In [9]:
from keras.datasets import cifar10
from keras.utils import np_utils 
from keras.models import Sequential 
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D 
from keras.optimizers import SGD, Adam, RMSprop 
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt 
from keras.preprocessing.image import ImageDataGenerator
import numpy as np 
import ssl

ssl._create_default_https_context=ssl._create_unverified_context


NUM_TO_AUGMENT=5 

# CIFAR_10 is a set of 60K images 32x32 pixels on 3 channels
IMG_CHANNELS = 3 
IMG_ROWS = 32 
IMG_COLS = 32 

#constant 
BATCH_SIZE = 512 
NB_EPOCH = 25 
NB_CLASSES = 10 
VERBOSE = 1 
VALIDATION_SPLIT = 0.2 
OPTIM = RMSprop() 

#load dataset 
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
#augmenting
print("Augmenting....")

datagen = ImageDataGenerator(rotation_range=40, width_shift_range=0.2, height_shift_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')
xtas, ytas = [], []
 
for i in range(X_train.shape[0]): 
    num_aug = 0 
    x = X_train[i] # (3, 32, 32) 
    x = x.reshape((1,) + x.shape) # (1, 3, 32, 32) 

for x_aug in datagen.flow(x, batch_size=1, save_to_dir ='preview', save_prefix='cifar', save_format='jpeg'): 
    if num_aug >= NUM_TO_AUGMENT: 
        break 
    xtas.append(x_aug[0]) 
    num_aug += 1


print('X_train shape:', X_train.shape) 
print(X_train.shape[0], 'train samples') 
print(X_test.shape[0], 'test samples')

# convert to categorical 
Y_train = np_utils.to_categorical(y_train, NB_CLASSES) 
Y_test = np_utils.to_categorical(y_test, NB_CLASSES) 

# float and normalization 
X_train = X_train.astype('float32') 
X_test = X_test.astype('float32') 
X_train /= 255 
X_test /= 255

# network 
model = Sequential() 
model.add(Conv2D(32, (3, 3), padding='same', input_shape=(IMG_ROWS, IMG_COLS, IMG_CHANNELS))) 
model.add(Activation('relu')) 
model.add(Conv2D(32, (3, 3), padding='same')) 
model.add(Activation('relu')) 
model.add(MaxPooling2D(pool_size=(2, 2))) 
model.add(Dropout(0.25)) 
model.add(Conv2D(64, (3, 3), padding='same')) 
model.add(Activation('relu')) 
model.add(Conv2D(64, 3, 3)) 
model.add(Activation('relu')) 
model.add(MaxPooling2D(pool_size=(2, 2))) 
model.add(Dropout(0.25)) 

model.add(Flatten()) 
model.add(Dense(512))
model.add(Activation('relu')) 
model.add(Dropout(0.5)) 
model.add(Dense(NB_CLASSES)) 
model.add(Activation('softmax'))

model.summary()

#fit the dataget 
datagen.fit(X_train)

# old train 
#model.compile(loss='categorical_crossentropy', optimizer=OPTIM, metrics=['accuracy']) 
#model.fit(X_train, Y_train, batch_size=BATCH_SIZE, epochs=NB_EPOCH, validation_split=VALIDATION_SPLIT, verbose=VERBOSE) 
#score = model.evaluate(X_test, Y_test, batch_size=BATCH_SIZE, verbose=VERBOSE) 
#print("Test score:", score[0]) 
#print('Test accuracy:', score[1])

#new train
model.compile(loss='categorical_crossentropy', optimizer=OPTIM, metrics=['accuracy'])

#checkpoints
checkpoint_filepath = 'keras_check'
cp_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)


history = model.fit_generator(datagen.flow(X_train, Y_train, batch_size=BATCH_SIZE), 
samples_per_epoch=X_train.shape[0], epochs=NB_EPOCH, verbose=VERBOSE, callbacks = [cp_callback]) 
score = model.evaluate(X_test, Y_test, batch_size=BATCH_SIZE, verbose=VERBOSE) 
print("Test score:", score[0]) 
print('Test accuracy:', score[1])


#save model 
model_json = model.to_json() 
open('cifar10_architecture.json', 'w').write(model_json) 
#And the weights learned by our deep network on the training 
model.save_weights('cifar10_weights.h5', overwrite=True)






Augmenting....
X_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:75: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3))`


Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_33 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
activation_49 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_50 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_25 (Dropout)         (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_35 (Conv2D)           (None, 16, 16, 64)       

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:113: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=25, verbose=1, callbacks=[<keras.ca..., steps_per_epoch=97)`


Epoch 1/25
97/97 [==============================] - 76s 786ms/step - loss: 2.1271 - accuracy: 0.2288
Epoch 2/25


C:\ProgramData\Anaconda3\lib\site-packages\keras\callbacks\callbacks.py:707: RuntimeWarning: Can save best model only with val_accuracy available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


97/97 [==============================] - 77s 790ms/step - loss: 1.9123 - accuracy: 0.3049
Epoch 3/25
97/97 [==============================] - 78s 806ms/step - loss: 1.7923 - accuracy: 0.3484
Epoch 4/25
97/97 [==============================] - 75s 771ms/step - loss: 1.7002 - accuracy: 0.3879
Epoch 5/25
97/97 [==============================] - 125s 1s/step - loss: 1.6296 - accuracy: 0.4138
Epoch 6/25
97/97 [==============================] - 150s 2s/step - loss: 1.5698 - accuracy: 0.4341
Epoch 7/25
97/97 [==============================] - 156s 2s/step - loss: 1.5261 - accuracy: 0.4538
Epoch 8/25
97/97 [==============================] - 155s 2s/step - loss: 1.4845 - accuracy: 0.4685
Epoch 9/25
97/97 [==============================] - 152s 2s/step - loss: 1.4470 - accuracy: 0.4844
Epoch 10/25
97/97 [==============================] - 151s 2s/step - loss: 1.4247 - accuracy: 0.4923
Epoch 11/25
97/97 [==============================] - 150s 2s/step - loss: 1.3882 - accuracy: 0.5033
Epoch 12/25
9

Your face is considered to be PII (Imperva, n.d.). After all, what could possibly be more identifiable to a person than their own face. PII, ethically, and now in many cases legally is to be treated with the highest degree of care. In fact, some places have gone as far as to codify this, e.g., GDPR. If a company starts to scrape images from public sources and then pairs that image with a name then all ideas of privacy have been breached – it would be easy for anyone with access to the data to identify you directly.

In Hill (2020) the story of ClearView AI is even worse as those images and names are then turned over to the cops. More than likely the police wouldn’t be able to secure that data without a warrant, but because they are using a 3rd party software they are in the clear. 

What about real-time facial recognition? Could the police use cameras to search citizens going about their day and then arrest anyone who matches another person with a warrant? Police typically need probable cause to make a stop, is looking similar to a wanted person enough?

Moreover, facial recognition isn’t flawless. Najibi (2020) shows that across 5 different systems, females with darker complexion have the lowest accuracy. Darker males are also less accurate than lighter males. These factors could cause more false positives leading to false arrests and detainments.

Hill, K. (2020, January 21). The secretive company that might end privacy as we know it. International New York Times, NA. https://link-gale-com.ezproxy.snhu.edu/apps/doc/A611773186/GIC?u=nhc_main&sid=bookmark-GIC&xid=602284b3

Imperva. What is personally identifiable information. (n.d.). https://www.imperva.com/ learn/data-security/personally-identifiable-information-pii

Najibi, A. (2020). Racial discrimination in face recognition technology. Science in the News. https://sitn.hms.harvard.edu/flash/2020/racial-discrimination-in-face-recognition-technology